In [1]:
import numpy as np
import scipy as sp
import pandas as pd
import rebound
import reboundx

## Set parameters

In [2]:
migration = "UNMigration" #"NoMigration" "UNMigration"
chemdist = "binary" #"binary" "linear" "no_overlap"

In [3]:
#Set locations for comets
def comets_loc(inner_lim, outer_lim, n_comets):
    comets = randomstate.uniform(low = inner_lim, high = outer_lim , size = n_comets)
    return comets

In [4]:
#Set specific random seed for each cluster node sim, used to replicate for later
seedinput = 100 #2 #13 #55

In [12]:
randomstate = np.random.RandomState(seedinput)
#np.random.seed(seed=seedinput)
filenum = seedinput

n_comets = 10
#Set timesteps (tau = planetary migration timescale recalculation, enc = stellar encounter samples)
tau_timestep_num = 100
#enc_timestep_num = 2 #always one more than (tmax - t_ce) / desired interval bewteeen steps
#Set interval at which rebound simulation snapshot is taken
sim_archive_interval = 5e5 #5e5

#Integrator used by simulation
sim_integrator = "mercurius" #"ias15" "whfast "mercurius"

#Set distance (in terms of Hill Radii) at which integrator switches to IAS15 to handle close encounters (for Mercurius)
RcriticalMerc = 6.

#Set distance (in AU) at which objects are removed from the sim
object_exit_distance = 1e99 #206266 is 1 pc #1e99 for no removals

if chemdist == "binary":
    a_comets = comets_loc(5.,45.,n_comets)
e_comets = comets_loc(0.15,0.15,n_comets)

#tmax is the total simulation time
#T_ce is the time at which the planetary bodies stop moving (but the comets will move)
tmax = 4.5e9 #1.2e6 #4.5e9
T_ce = 1e6

#Set the semi-major axis and eccentricity timescales for Uranus and Neptune only
if migration == "UNMigration":
    a_i_list = [5.2044, 9.5826, 17.5, 27.5]
    e_i_list = [0.0489, 0.0565, 0.2, 0.3]

a_f_list = [5.2044, 9.5826, 19.2184, 30.1104]
e_f_list = [0.0489, 0.0565, 0.0463, 0.00946]

## Filename Strings

In [13]:
if sim_integrator == "ias15":
    integrator_filelabel = "_IAS15_"
elif sim_integrator == "mercurius":
    integrator_filelabel = "_Rcrit"+str(RcriticalMerc)+"_"

## Tau

In [14]:
#Define Tau values (for planetary migration)
def Tau(x_i,x_f,T_ce,t):
    Tau = (T_ce/(2.*np.log(x_i/x_f)))/(1.-t/(T_ce))
    return abs(float(Tau))

## Integration function

In [15]:
def do_calc(sim,t):
    while True:
        try:
            sim.integrate(t)
            break
        #Remove all stars that exceed r_max
        except rebound.Escape as error:
            #print(error)
            for j in range(sim.N):
                p = sim.particles[j]
                d2 = p.x*p.x + p.y*p.y + p.z*p.z
                if d2 > sim.exit_max_distance**2:
                    index = j # cache index rather than remove here since our loop would go beyond end of particles array
            sim.remove(index=index)
    #orbits = sim.calculate_orbits()
    #orbits_list.append(orbits)    
    return sim

## Function that creates the simulation

In [16]:
def simulation_create():
    #Call simulations stuffs
    sim = rebound.Simulation()
    sim.units = ('yr', 'AU', 'Msun')

    #Retrieve planet data from file
    try:
        sim = rebound.Simulation.from_file("./solarsystem_"+migration+".bin")
    except:
        sim.add('sun')
        sim.add('jupiter')
        sim.add('saturn')
        if migration == "UNMigration":
            sim.add(m=4.3645e-5,a=a_i_list[2], e=e_i_list[2], inc=0., Omega=0, omega=0, f=0 )
            sim.add(m=5.1483e-5,a=a_i_list[3], e=e_i_list[3], inc=0., Omega=0, omega=0, f=0 )
        elif migration == "NoMigration":
            sim.add('uranus')
            sim.add('neptune')
        sim.save("./solarsystem_"+migration+".bin")
    
    sim.integrator = sim_integrator
    sim.collision = "mercurius"
    sim.collision_resolve = "merge"
    sim.collision_resolve_keep_sorted = 1
    #sim.boundary = "open"

    if ((sim.integrator == "whfast") | (sim.integrator == "mercurius")):
        sim.dt =  sim.particles[1].P/40.

    sim.ri_mercurius.rcrit = RcriticalMerc

    #Call reboundx for additional forces
    rebx = reboundx.Extras(sim)
    #Add in force that determines planet migration 
    #rebx.add("modify_orbits_forces")
    mod_effect = rebx.add("modify_orbits_direct")
    mod_effect.params["p"] = 0.

    for i in xrange(0,n_comets): 
        rand = randomstate.random_sample()*2*np.pi
        sim.add(a=a_comets[i], e=e_comets[i], inc=0., Omega=0, omega=rand, f=rand)
    
    sim.move_to_com()
    sim.exit_max_distance = object_exit_distance

    #Set simulationarchive to reload later in case sim is interrupted
    sim.automateSimulationArchive("sim_save_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem.bin",
                                  interval=sim_archive_interval,deletefile=False)
    
    #Timesteps at which tau is recalculated (note that perturbation sampling also occurs here at the same timestep)
    tau_times = np.linspace(0, T_ce, tau_timestep_num,endpoint=False)

    #Simulate planet migration
    for time in tau_times:
        if migration == "UNMigration":
            sim.particles[3].params["tau_a"] = Tau(a_i_list[2],a_f_list[2],T_ce,sim.t)
            sim.particles[4].params["tau_a"] = Tau(a_i_list[3],a_f_list[3],T_ce,sim.t)
            sim.particles[3].params["tau_e"] = -Tau(e_i_list[2],e_f_list[2],T_ce,sim.t)
            sim.particles[4].params["tau_e"] = -Tau(e_i_list[3],e_f_list[3],T_ce,sim.t)
        elif migration == "NoMigration":
            sim.particles[3].params["tau_a"] = np.inf
            sim.particles[4].params["tau_a"] = np.inf
            sim.particles[3].params["tau_e"] = -np.inf
            sim.particles[4].params["tau_e"] = -np.inf
        else:
            pass
        
        sim = do_calc(sim,time)
        print("t = ",sim.t)
            
    print("t > T_ce")
    #print(str(sim_name)+" Working on t > T_ce")
    sim.particles[3].params["tau_a"] = np.inf #after time reaches T_ce, tau_a becomes infinity (time for planets to move additional distance is infinite, i.e. plaents don't move)
    sim.particles[4].params["tau_a"] = np.inf
    sim.particles[3].params["tau_e"] = -np.inf
    sim.particles[4].params["tau_e"] = -np.inf

    #With simulationarchive saving structure
    sim = do_calc(sim,tmax)
        
    #Delete sim
    #del sim
    
    #Get orbital parameters at each rebound snapshot
    sa = rebound.SimulationArchive("sim_save_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem.bin")
    
    orbits_list = []
    for i, sim in enumerate(sa):
        orbits_list.append(sim.calculate_orbits())

    return sim_name, orbits_list

## Function that loads sim

In [17]:
def simulation_load():
    #load Saved Simulation archive
    print("sim_save_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem.bin")
    sa = rebound.SimulationArchive("sim_save_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem.bin")
    
    print("Number of snapshots: %d" % len(sa))
    print("Time of first and last snapshot: %.1f, %.1f" % (sa.tmin, sa.tmax))
    sim_saved = sa[-1]
    
    sim_saved.integrator = sim_integrator
    sim_saved.collision = "mercurius"
    sim_saved.collision_resolve = "merge"
    sim_saved.collision_resolve_keep_sorted = 1
    #sim_saved.boundary = "open"

    if ((sim_saved.integrator == "whfast") | (sim_saved.integrator == "mercurius")):
        sim_saved.dt =  sim_saved.particles[1].P/40.  

    sim_saved.ri_mercurius.rcrit = RcriticalMerc

    rebx = reboundx.Extras(sim_saved)
    mod_effect = rebx.add("modify_orbits_direct")
    mod_effect.params["p"] = 0.  
    
    sim_saved.move_to_com()
    sim_saved.exit_max_distance = object_exit_distance

    #Set simulationarchive to reload later in case sim is interrupted
    sim_saved.automateSimulationArchive("sim_save_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem.bin",
                                  interval=sim_archive_interval,deletefile=False)
    
    #Simulation assumed to have already gotten past 1e6 yrs before walltime is exceeded on cluster
    sim_saved.particles[3].params["tau_a"] = np.inf #after time reaches T_ce, tau_a becomes infinity (time for planets to move additional distance is infinite, i.e. plaents don't move)
    sim_saved.particles[4].params["tau_a"] = np.inf
    sim_saved.particles[3].params["tau_e"] = -np.inf
    sim_saved.particles[4].params["tau_e"] = -np.inf
        
    sim_saved = do_calc(sim_saved,tmax)

    #Delete sim
    #del sim_saved

    #Get orbital parameters at each rebound snapshot
    sa_1 = rebound.SimulationArchive("sim_save_"+migration+"_"+str(filenum)+integrator_filelabel+chemdist+"chem.bin")
    
    orbits_list = []
    for i, sim_saved in enumerate(sa_1):
        orbits_list.append(sim_saved.calculate_orbits())
        
    for orbits in orbits_list:
        i = 0
        for orbit in orbits:
            print(str(i)+", "+str(orbit.a)+", "+str(orbit.e))
            i += 1
        print("")

In [ ]:
try:
    simulation_load()
except ValueError:
    import time
    t0 = time.time()
    print("Load error, creating simulation ...")
    sim_name, orbits_list = simulation_create()

    for orbits in orbits_list:
        i = 0
        for orbit in orbits:
            print(str(i)+", "+str(orbit.a)+", "+str(orbit.e))
            i += 1
        print("")
    t1 = time.time()
    total = t1 - t0
    print(total)

sim_save_UNMigration_100_Rcrit6.0_binarychem.bin
Load error, creating simulation ...
t =  0.0
t =  10000.0
t =  20000.0
t =  30000.0
t =  40000.0
t =  50000.0
t =  60000.0
t =  70000.0
t =  80000.0
t =  90000.0
t =  100000.0
t =  110000.0
t =  120000.0
t =  130000.0
t =  140000.0
t =  150000.0
t =  160000.0
t =  170000.0
t =  180000.0
t =  190000.0
t =  200000.0
t =  210000.0
t =  220000.0
t =  230000.0
t =  240000.0
t =  250000.0
t =  260000.0
t =  270000.0
t =  280000.0
t =  290000.0
t =  300000.0
t =  310000.0
t =  320000.0
t =  330000.0
t =  340000.0
t =  350000.0
t =  360000.0
t =  370000.0
t =  380000.0
t =  390000.0
t =  400000.0
t =  410000.0
t =  420000.0
t =  430000.0
t =  440000.0
t =  450000.0
t =  460000.0
t =  470000.0
t =  480000.0
t =  490000.0
t =  500000.0
t =  510000.0
t =  520000.0
t =  530000.0
t =  540000.0
t =  550000.0
t =  560000.0
t =  570000.0
t =  580000.0
t =  590000.0
t =  600000.0
t =  610000.0
t =  620000.0
t =  630000.0
t =  640000.0
t =  650000.0
t =  